## Step 3 : shift the expansion M2M

In the step 2 we grouped the sources in the same cell; but when the number
of particles is large we need to build a hierarchical structure to group the particles, and make the code more efficient.

We already know that the multipole of a cell only depends on the sources in that cell and the center of that cell. (the multipole is a "property" of a cell).

Due to this "property", we are going to be able to evaluate the multipole of a parent cell given the multipole of its child cells. 

Here is how look some terms of the multipole expansion of a child cell $$c:

\begin{equation}
\begin{split}
M_c &= \sum_{j=0}^{N_c} m_j\\
D_{c_x} &= \sum_{j=0}^{N_c} m_j\,( x_c - x_j)\\
Q_{c_{xx}} &= \sum_{j=0}^{N_c} \frac{1}{2}m_j\, ( x_c - x_j)^2 \\
Q_{c_{xy}} &= \sum_{j=0}^{N_c} \frac{1}{2}m_j\, (x_c-x_j)(y_c-y_j) \\
\end{split}
\end{equation}

where $N_c$ is the number of sources in the child cell, $x_c$ is the cell's center, $M$, $D$, $Q$ are respectively monopole, dipole and quadrupole. (here only showing some terms, the other ones are similar)

We will expand the particles from a *child* cell to a *parent* one, we will need to shift the center of expansion from ($x_c$, $y_c$, $z_c$) to ($x_p$, $y_p$, $z_p$). The expansion of the particles of a parent cell $p$ with 8 child cells:

(We present some terms of the expansion, the other ones are similar.)

\begin{equation}
\begin{split}
M_p &= \sum_{j=0}^N m_j = \sum_{c=0}^7 M_c\\
D_{p_x} &= \sum_{j=0}^N m_j\,( x_p - x_j) = \sum_{c=0}^7 \left[\sum_{j=0}^{N_c} m_j\,(x_c-x_j) + ( x_p-x_c)\sum_{j=0}^{N_c} m_j\right]\\
&= \sum_{c=0}^{7} \left[D_{c_x} + (x_p-x_c)\,M_c\right] \\
Q_{p_{xx}} &= \sum_{j=0}^N \frac{1}{2}m_j \,(x_p - x_j)^2 = \sum_{c=0}^7 \left[\sum_{j=0}^N \frac{1}{2}m_j\left[(x_c-x_j)+(x_p-x_c)\right]^2 \right]\\
&= \sum_{c=0}^7 \left[\sum_{j=0}^N \frac{1}{2}m_j (x_c - x_j)^2 + (x_p-x_c) \sum_{j=0}^N m_j(x_c-x_j) + \frac{(x_p-x_c)^2}{2} \sum_{j=0}^N m_j \right]\\
&= \sum_{c=0}^7 \left[Q_{c_{xx}} + (x_p-x_c)D_c + \frac{(x_p-x_c)^2}{2}M_c \right]\\
Q_{p_{xy}} &= \sum_{j=0}^{N} \frac{1}{2}m_j\, (x_p-x_j)(y_p-y_j) \\
&= \sum_{c=0}^7 \left[\sum_{j=0}^{N} \frac{1}{2}m_j(x_c-x_j)(y_c-y_j) + \frac{(x_p-x_c)}{2}\sum_{j=0}^{N}m_j(y_c-y_j) \right.\\
&+\left. \frac{(y_p-y_c)}{2}\sum_{j=0}^{N}m_j(x_c-x_j) + \frac{(x_p-x_c)(y_p-y_c)}{2}\sum_{j=0}^{N}m_j\right]\\
&= \sum_{c=0}^7 \left[ Q_{c_{xy}} +\frac{(x_p-x_c)}{2}D_{c_y} +\frac{(y_p-y_c)}{2}D_{c_x} + \frac{(x_p-x_c)(y_p-y_c)}{2}M_c \right]
\end{split}
\end{equation}

Each multipole term $M_p$, $D_p$, $Q_p$ in parent cell can be expressed by a summation of its child's multipole terms $M_c$, $D_c$, $Q_c$. Therefore, we refer this expansion shifting process as **Multipole-to-Multipole** or **M2M**

In [15]:
import numpy
from treecode_helper import Particle, distance, l2_err, plot_err
from matplotlib import pyplot, rcParams
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D

# customizing plot parameters
rcParams['figure.dpi'] = 100
rcParams['font.size'] = 16
rcParams['font.family'] = 'StixGeneral'

In [16]:
n = 50     # number of particles
m = 1./n    # mass

# initialize sources and targets
source_coords = numpy.random.random((n, 3)).tolist()
target_coords = (-numpy.random.random((n, 3))).tolist()
sources = [ Particle(coord, m=m) for coord in source_coords ]
targets = [ Particle(coord, m=m) for coord in target_coords ]

In [17]:
class Cell():
    '''
    The class for a cell
    
    Attributes:
    -----------
        x, y, z: x, y and z coordinates of the cell's center
        multipole: the array that contains the multipoles of a cell
    '''
    
    def __init__(self, x=None, y=None, z=None):
        
        #coordinates
        self.x = x
        self.y = y
        self.z = z
        
        #multipole array
        self.multipole = numpy.zeros(10)

In [19]:
#Creating a parent cell
parent = Cell(0.5, 0.5, 0.5) #the inputs are the coordinates of the cell's center

#Creating the 8 child-cells
octant = numpy.arange(8)

#Remember in python 3 the // is integer division 
x = (octant%2) * parent.x + parent.x/2.
y = ((octant//2)%2) * parent.y + parent.y/2.
z = ((octant//4)%2) * parent.z + parent.z/2.

#Group of child cells
children = [ Cell(x[i], y[i], z[i]) for i in range(8) ]
#print(children[0].x)

0.25


In [12]:
#I was doing this to understand the previous cell, it seems it is a
#pretty common way of indexing and locating coordinates in a 3D structure. 
'''octant = numpy.arange(8)

x = octant%2
y = (octant//2)#%2
z = (octant//4)#%2


print(octant)
#print(x)
print(y)
#print(z)
''';

In [ ]:
def P2M(sources, parent, children):
    '''
    Calculate the multipoles for the child cells.
    
    Arguments:
    ----------
        sources: list of source objects in Particle class
        parent: parent cell object
        children: list of child cell objects 
    '''
    
    for source in sources:
        
        octant = (source.x > parent.x) + ((source.y > parent.y) << 1) + ((source.z > parent.z) << 2) 
        
        dx = children[octant].x - source.x
        dy = children[octant].y - source.y
        dz = children[octant].z - source.z
        
        children[octant].multipole += source.m * numpy.array((1, dx, dy, dz, 0.5*dx**2, 0.5*dy**2, 0.5*dz**2,\
                                                              0.5*dx*dy, 0.5*dy*dz, 0.5*dz*dx)) 

In [13]:
#Notes to understand the function P2M
#We can sum logic operations, if the statement is True then the
#value assigned to that operation is 1, if it is False is 0 then,
#for example (3<1)+(3<1) = 0 , (3>1)+(3<1) = 1, (3>1)+(3>1) = 2

#Doing this to understand shifting "<<"
'''
print(0<<1)  
print(1<<1)
print(2<<1)
print('-----')
print(0<<2)
print(1<<2)
print(2<<2)
''';

x << y
    
Returns x with the bits shifted to the left by y places (and new bits on the right-hand-side are zeros). This is the same as multiplying x by 2**y. 

x >> y

Returns x with the bits shifted to the right by y places. This is the same as //'ing x by 2**y. 

[1 2 3]
[2 3 1]
